In [1]:
import scipy.io
import os
from PIL import Image
from matplotlib import pyplot as plt
import numpy as np
import torch
import pytorch_lightning as ptl
from weatherGan.models.Discriminators import WeatherDiscriminator

In [2]:
import tqdm
class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self,  data_dir, transform=None, target_transform=None,mode='train'):
        super(CustomImageDataset,self).__init__()
        imgs, segs, labels, targets = self.__load_images(data_dir,mode)
        self.imgs = imgs
        self.segs = segs
        self.labels = labels
        self.targets = targets
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.imgs)
    def img_to_seg(self,image_name):
        idx = image_name.index('_')
        seg_name = image_name[:idx]+'_mask'+image_name[idx:-3]+'mat'
        return seg_name
        
    
    def __load_images(self,path,mode='train'):
        path_img, path_mask = path+f'{mode}_images/', path+f'{mode}_mats/' 
        images = [img for img in os.listdir(path+f'{mode}_images/')]
        imgs, segs, labels, targets = [], [], [], []
        for img in tqdm.tqdm_notebook(images):
            img_path = path_img + img
            seg_path = path_mask + self.img_to_seg(img)
            img_array = np.array(Image.open(img_path))
            seg_array = np.array(scipy.io.loadmat(seg_path)['seg_mask'])
            imgs.append(img_array)
            segs.append(seg_array)
            label = np.array([ 'sunny' in img ])
            labels.append(label)
            targets.append(1-label)
        return imgs, segs, labels, targets

    def __getitem__(self, idx):
        image = self.imgs[idx]
        label = self.labels[idx]
        target = self.targets[idx]
        if self.transform:
            image = self.transform(image)
            
        if self.target_transform:
            label = self.target_transform(label)
            target = self.target_transform(target)
            
        return torch.tensor(image).permute(2,0,1), torch.tensor(self.segs[idx]), label, target

In [3]:
PATH = 'datasets/dataset/'
train_dataset = CustomImageDataset(data_dir=PATH,transform= lambda x : x/255,target_transform= lambda x : torch.nn.functional.one_hot( torch.tensor(int(x)), 2)
)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/8000 [00:00<?, ?it/s]

In [4]:
val_dataset = CustomImageDataset(data_dir=PATH,mode='test',transform= lambda x : x/255,target_transform= lambda x : torch.nn.functional.one_hot( torch.tensor(int(x)), 2)
)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/2000 [00:00<?, ?it/s]

In [5]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset, batch_size = 32,num_workers=8,shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = 1,num_workers=8,shuffle = False)

In [6]:
for img, seg, _ ,_ in train_loader:
    break

In [7]:
import torchmetrics

In [8]:
class WeatherClassifier(ptl.LightningModule):
    def __init__(self,num_labels=2,lr=2e-5,**kwargs):
        super(WeatherClassifier,self).__init__()
        self.num_labels = num_labels
        self.lr = lr
        self.model = WeatherDiscriminator(num_labels=num_labels)
        self.ce = torch.nn.CrossEntropyLoss()
        self.acc = torchmetrics.functional.accuracy
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(lr=self.lr,params=self.parameters() )
        return optimizer
    def forward(self,image):
        pred = self.model(image)
        return pred 
    
    def training_step(self,batch, batch_idx):
        image, _, label, _ = batch
        batch_size, _, h, w = image.shape
        prediction = self(image)
        loss = self.ce(prediction, torch.argmax(label,dim=-1).long())
        accuracy = self.acc(prediction, torch.argmax(label,dim=-1).long() )
        self.log("loss",loss,prog_bar=True,on_step=True)
        self.log("accuracy", accuracy, prog_bar=True,on_step = True)
        return loss
    def validation_step(self,batch, batch_idx):
        image, _, label, _ = batch
        batch_size, _, h, w = image.shape
        prediction = self(image)
        loss = self.ce(prediction, torch.argmax(label,dim=-1).long())
        accuracy = self.acc(prediction, torch.argmax(label,dim=-1).long() )
        self.log("val_loss",loss,prog_bar=True,on_step=True)
        self.log("val_accuracy", accuracy, prog_bar=True,on_step = True)
        return loss
    def test_step(self,batch, batch_idx):
        image, _, label, _ = batch
        batch_size, _, h, w = image.shape
        prediction = self(image)
        loss = self.ce(prediction, torch.argmax(label,dim=-1).long())
        accuracy = self.acc(prediction, torch.argmax(label,dim=-1).long() )
        self.log("test_loss",loss,prog_bar=True,on_step=True)
        self.log("test_accuracy", accuracy, prog_bar=True,on_step = True)
        return loss
        

In [9]:
model = WeatherClassifier.load_from_checkpoint('checkpoints/weatherClassifier/epoch=9-step=2499.ckpt').double()

In [10]:
model.to(0)

WeatherClassifier(
  (model): WeatherDiscriminator(
    (featureExtractor): FeatureExtractor(
      (convBlock1): ConvBlock(
        (conv): Conv2d(3, 3, kernel_size=(32, 32), stride=(2, 2), padding=(1, 1))
        (activation): ReLU()
        (pool): MaxPool2d(kernel_size=(2, 2), stride=1, padding=0, dilation=1, ceil_mode=False)
        (layernorm): LayerNorm((3, 135, 135), eps=1e-05, elementwise_affine=True)
      )
      (convBlock2): ConvBlock(
        (conv): Conv2d(3, 3, kernel_size=(32, 32), stride=(2, 2), padding=(1, 1))
        (activation): ReLU()
        (pool): MaxPool2d(kernel_size=(2, 2), stride=1, padding=0, dilation=1, ceil_mode=False)
        (layernorm): LayerNorm((3, 52, 52), eps=1e-05, elementwise_affine=True)
      )
      (convBlock3): ConvBlock(
        (conv): Conv2d(3, 3, kernel_size=(32, 32), stride=(2, 2), padding=(1, 1))
        (activation): ReLU()
        (pool): MaxPool2d(kernel_size=(2, 2), stride=1, padding=0, dilation=1, ceil_mode=False)
        (layer

In [11]:
proba_0, proba_1, predicted_labels, true_labels = [], [], [], []
for image, _, label, _ in tqdm.notebook.tqdm(val_loader):
    image = image.to(torch.device(0))
    pred = torch.softmax(model(image),dim=-1)
    pred = pred.detach().cpu()
    predicted_label = torch.argmax(pred,dim=-1)[0]
    predicted_labels.append(predicted_label.numpy())
    true_labels.append(torch.argmax(label,dim=-1)[0].numpy())
    proba_0.append(pred[0,0].numpy())
    proba_1.append(pred[0,1].numpy())
    del image
    

  0%|          | 0/2000 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448265233/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [12]:
import pandas as pd

df = pd.DataFrame.from_dict({'0':proba_0,'1':proba_1,'label':predicted_labels,'true_label':true_labels})

In [14]:
df.to_csv('weather_predictions.csv')

In [17]:
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
checkpoint_callback = ModelCheckpoint(dirpath='checkpoints/weatherClassifier/',monitor='val_loss',mode='min')
wandbLogger = WandbLogger(project='weatherGan',name='weatherClassifier')

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:446: UserWarning: Checkpoint directory checkpoints/weatherClassifier/ exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


In [18]:
trainer = ptl.Trainer(gpus=[0],max_epochs=10,callbacks=[checkpoint_callback],
                     logger=wandbLogger,log_every_n_steps = 5
                     )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [35]:
trainer.fit(model, train_loader,val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
wandb: Currently logged in as: adaminho (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade



  | Name  | Type                 | Params
-----------------------------------------------
0 | model | WeatherDiscriminator | 190 K 
1 | ce    | CrossEntropyLoss     | 0     
-----------------------------------------------
190 K     Trainable params
0         Non-trainable params
190 K     Total params
0.762     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: -1it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/properties.py:306: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_progress_bar_dict()` in `LightingModule`.
  UserWarning,


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [20]:
model = WeatherClassifier.load_from_checkpoint('checkpoints/weatherClassifier/epoch=9-step=2499.ckpt').double()

In [21]:
trainer.test(model,val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.8169999718666077,
 'test_accuracy_epoch': 0.8169999718666077,
 'test_loss': 0.4102165102958679,
 'test_loss_epoch': 0.4102165102958679}
--------------------------------------------------------------------------------


[{'test_loss': 0.4102165102958679,
  'test_loss_epoch': 0.4102165102958679,
  'test_accuracy': 0.8169999718666077,
  'test_accuracy_epoch': 0.8169999718666077}]